# HMM Simulation

In [1]:
import numpy as np

> <p>Code:</p>
> <pre>
>   Burger : 0,
>   Pizza : 1,
>   Hotdog : 2
> </pre>

In [3]:
state = {
    0 : "Burger",
    1 : "Pizza",
    2 : "Hotdog"
}
state

{0: 'Burger', 1: 'Pizza', 2: 'Hotdog'}

### Transition Matrix
A_ij = P(X_n = j | X_n-1 = i)

In [4]:
A = np.array([[0.2, 0.6, 0.2], [0.3, 0.0, 0.7], [0.5, 0.0, 0.5]])
A

array([[0.2, 0.6, 0.2],
       [0.3, 0. , 0.7],
       [0.5, 0. , 0.5]])

### Random Walk on Markov Chain

In [5]:
n = 15
start_state = 0
print(state[start_state], "--->", end=" ")
prev_state = start_state

while n-1:
   curr_state = np.random.choice([0, 1, 2], p=A[prev_state])
   print(state[curr_state], "--->", end=" ")
   prev_state = curr_state
   n -= 1
print("stop")

Burger ---> Pizza ---> Hotdog ---> Burger ---> Pizza ---> Hotdog ---> Hotdog ---> Burger ---> Hotdog ---> Hotdog ---> Hotdog ---> Hotdog ---> Hotdog ---> Burger ---> Pizza ---> stop


### Find Transient State

Here are three methods to find transient state probibility.

### Approach 1 : Monte Carlo

In [6]:
steps = 10**6
start_state = 0
curr_state = start_state
pi = np.array([0, 0, 0])
pi[start_state] = 1

i = 0
while i<steps:
    curr_state = np.random.choice([0,1,2], p=A[curr_state])
    pi[curr_state]+=1
    i +=1

print("π = ", pi/steps)

π =  [0.352042 0.211081 0.436878]


### Approach 2 : Repeated Matrix Multiplication

In [7]:
steps = 10**3
A_n = A

i=0
while i<steps:
    A_n =  np.matmul(A_n, A)
    i+=1

print("A^n = \n", A_n, "\n")
print("π = ", A_n[0])

A^n = 
 [[0.35211268 0.21126761 0.43661972]
 [0.35211268 0.21126761 0.43661972]
 [0.35211268 0.21126761 0.43661972]] 

π =  [0.35211268 0.21126761 0.43661972]


### Approach 3 : Finding Left Eigen Vectors

Just need to find the solution of $\pi \cdot \mathbb{A} = \pi$.

In [8]:
import scipy.linalg
values, left = scipy.linalg.eig(A, right = False, left = True)

print("left eigen vectors = \n", left, "\n")
print("eigen values = \n", values)

left eigen vectors = 
 [[-0.58746336+0.j         -0.16984156-0.35355339j -0.16984156+0.35355339j]
 [-0.35247801+0.j          0.67936622+0.j          0.67936622-0.j        ]
 [-0.72845456+0.j         -0.50952467+0.35355339j -0.50952467-0.35355339j]] 

eigen values = 
 [ 1.  +0.j        -0.15+0.3122499j -0.15-0.3122499j]


In [9]:
pi = left[:,0]
pi_normalized = [(x/np.sum(pi)).real for x in pi]
pi_normalized

[0.3521126760563379, 0.21126760563380292, 0.43661971830985913]

### Exercise of finding P(Pizza --> Hotdog --> Hotdog --> Burger) = ?
=> P(X_0 = Pizza, X_1 = Hotdog, X_2 = Hotdog, X_3 = Burger)

=> P(X_0 = Pizza) P(X_1 = Hotdog | X_0 = Pizza) P(X_2 = Hotdog | X_1 = Hotdog) P(X_3 = Burger | X_2 = Hotdog)

In [10]:
def find_prob(seq, A, pi):
   start_state = seq[0]
   prob = pi[start_state]
   prev_state, curr_state = start_state, start_state
   for i in range(1, len(seq)):
      curr_state = seq[i]
      prob *= A[prev_state][curr_state]
      prev_state = curr_state
   return prob

print(find_prob([1, 2, 2, 0], A, pi_normalized))

0.036971830985915506
